# CAI Lab Session 2: Intro to ElasticSearch

In this session you will learn:

- a few basics of the `ElasticSearch` database
- how to index a set of documents and how to ask simple queries about these documents
- how to do this from `Python`
- based on the previous, you will compute the boolean and tf-idf matrix for the toy corpus used in class

## 1. ElasticSearch

[ElasticSearch](https://www.elastic.co/) is a _NoSQL/document_ database with the capability of indexing and searching text documents. As a rough analogue, we can use the following table for the equivalence between ElasticSearch and a more classical relational database:

| Relational DB | ElasticSearch |
|---|---|
| Database | Index |
| Table | Type |
| Row / record | Document |
| Column | Field |

An index can be thought of as an optimized collection of documents and each document is a collection of fields, which are the key-value pairs that contain your data.

`ElasticSearch` is a pretty big beast with many options. Luckily, there is much documentation, a few useful links are:

- Here is the [full documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html)
- Intros you may want to have a look at: 
    - https://medium.com/expedia-group-tech/getting-started-with-elastic-search-6af62d7df8dd
    - http://joelabrahamsson.com/elasticsearch-101
- You found another one that you liked? Let us know. 

## 2. Running ElasticSearch

First you will need to install `ElasticSearch` following instructions in their [documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html).

This database runs as a web service in a machine and can be accessed using a REST web API; however we will interact with the database through its python libraries `elasticsearch-py` and `elasticsearch-dsl`, so you will need to install these as well.  You can run `ElasticSearch` by typing from the command-line prompt:

```
$ <path_to_elasticsearch_bin>/elasticsearch &
```



After a few seconds (and a lot of logging) the database will be up and running; you may need to hit return for the prompt to show up. To test whether `ElasticSearch` is working execute the code in the cell below. __The database needs to be running throughout the execution of this script, otherwise you will get a connection error.__

In [1]:
from pprint import pprint

In [2]:
import requests

try:
    resp = requests.get('http://localhost:9200/')
    pprint(resp.content)
    
except Exception:
    print('elasticsearch is not running')

elasticsearch is not running


If `ElasticSearch` is working you will see an answer from the server; otherwise you will see a message indicating that it is not running. You can try also throwing the URL http://localhost:9200 to your browser; you should get a similar answer.

**In version 8 they introduced enhanced security, which may give you trouble when executing the code here, to deal with this you can either install an earlier version (7 or older) or turn off security settings in their `config/elasticsearch.yml` config file (just set to _false_ everything concerning the security options).** Since we are using the database in offline, local mode this should not be a problem.

Also, you should run this script locally in your machine, if you use Google Collab or similar this is not going to work because elasticsearch should be running on the machine where the script is being executed.

## 3. Indexing and querying

`ElasticSearch` is a database that allows storing documents (tables do not need a predefined schema as in relational databases). Text in these documents can be processed so the queries extend beyond exact matches allowing complex queries, fuzzy matching and ranking documents respect to the actual match. 

These kinds of databases are behind search engines like Google Search or Bing.

There are different ways of operating with ElasticSearch. It is deployed esentially as a web service with a REST API, so it can be accessed basically from any language with a library for operating with HTTP servers.

We are going to use two python libraries for programming on top of ElasticSearch: `elasticsearch` and `elasticsearch-dsl`. Both provide access to ElasticSearch functionalities hiding and making more programming-friendly the interactions, the second one is more convenient for configurating and searching. Make sure both python libraries are installed to proceed with this session.

In [3]:
!pip3 install elasticsearch --user
!pip3 install elasticsearch-dsl --user


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


We are only going to see the essential elements for developing the session but feel free to learn more.

To interact with ElasticSearch with need a client object of type `Elasticsearch`.

In [4]:
from elasticsearch import Elasticsearch

client = Elasticsearch("http://localhost:9200", timeout=1000)

C:\Users\Carlos Arbonés\AppData\Local\Temp\ipykernel_8728\2120730465.py:3: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  client = Elasticsearch("http://localhost:9200", timeout=1000)


With this client you have a connection for operating with Elastic search. Now we will create an index. There are index operations in each library, but the one in `elasticseach-dsl` is simpler to use.

In [5]:
from elasticsearch_dsl import Index

index = Index('test', using=client)  # if it does not exist, it is created; if it does exist, then it connects

First we will need some text to index, for testing purposes we are going to use the python library `loremipsum`. We will need to install it first if it is not installed already, uncomment the code in next cell if you need to install the library

In [6]:
!pip install lorem --user  # Restart the kernel if you are not able to import the library in the next cell

ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Now we create some random paragraphs

In [7]:
import lorem

texts = [lorem.paragraph() for _ in range(10)]
print(len(texts))
print(texts[0])

10
Consectetur neque consectetur dolor velit modi. Sit consectetur numquam etincidunt labore modi. Ut quaerat neque etincidunt. Etincidunt numquam quiquia ipsum. Labore ipsum dolor modi dolorem quaerat. Sit voluptatem tempora consectetur non quisquam. Adipisci modi voluptatem tempora est.


Now we can index the paragraphs in ElasticSearch using the `index` method. The document is passed as a python dictionary with the `document` parameter. The keys of the dictionary will be the fields of the document, in this case we well have only one (`text`) -- here, we use this tag but could use anything we wanted to.

In [8]:
for t in texts:
    client.index(index='test', document={'text': t})
    print(f'Indexing new text: {t[:70]} ...')

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: ProtocolError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))))

In case we want to get all docs in the index, we can do the following:

In [ ]:
# get all docs in index 'test'
resp = client.search(index="test", query={"match_all": {}})

# print them
print(f"Got {resp['hits']['total']['value']} hits:")
for hit in resp['hits']['hits']:
    pprint(hit["_source"])

Got 10 hits:
{'text': 'Magnam labore amet dolore est quisquam amet adipisci. Numquam sed '
         'eius magnam. Tempora consectetur sed est neque modi aliquam labore. '
         'Dolor numquam eius sed voluptatem amet. Est neque ut numquam '
         'etincidunt. Eius aliquam consectetur ipsum eius amet. Ipsum quisquam '
         'porro modi eius. Eius est quisquam dolore dolore labore adipisci '
         'consectetur. Quiquia quisquam eius ipsum magnam sed ipsum.'}
{'text': 'Consectetur etincidunt non magnam consectetur voluptatem eius. '
         'Quiquia magnam sit dolore est quisquam. Sed voluptatem ipsum ut sed '
         'porro. Sed ut velit dolore quaerat. Labore porro non consectetur. '
         'Quaerat porro labore non quisquam amet sit numquam. Adipisci labore '
         'amet magnam velit neque labore adipisci.'}
{'text': 'Ipsum consectetur velit sed ipsum numquam modi. Est labore non ipsum '
         'est. Sed dolorem porro neque ut ut adipisci. Modi consectetur amet '
 

We can also search for documents that contain a given keyword:

In [ ]:
from elasticsearch_dsl import Search

# the following search query specifies the field where we want to search
s_obj = Search(using=client, index='test')
sq = s_obj.query('match', text='non')
resp = sq.execute()

print(f'Found {len(resp)} matches.')

for hit in resp:
    print(f'\nID: {hit.meta.id}\nText: {hit.text}')

Found 8 matches.

ID: 6GY1mIoB1aJZhB_GIOs0
Text: Velit ut numquam dolore sit dolorem quisquam. Sit consectetur neque velit est consectetur non voluptatem. Ipsum dolorem dolorem voluptatem amet velit numquam quaerat. Sed labore labore non. Ut tempora etincidunt sed ipsum non modi consectetur.

ID: 5GY1mIoB1aJZhB_GH-uq
Text: Consectetur etincidunt non magnam consectetur voluptatem eius. Quiquia magnam sit dolore est quisquam. Sed voluptatem ipsum ut sed porro. Sed ut velit dolore quaerat. Labore porro non consectetur. Quaerat porro labore non quisquam amet sit numquam. Adipisci labore amet magnam velit neque labore adipisci.

ID: 52Y1mIoB1aJZhB_GIOsS
Text: Numquam etincidunt magnam sed aliquam amet. Dolorem non labore quaerat voluptatem. Est eius non dolorem consectetur voluptatem sed sit. Modi quiquia ipsum neque labore velit ipsum etincidunt. Dolor ut etincidunt magnam. Sit sed ipsum est modi. Consectetur ut eius ut. Dolorem sit voluptatem neque.

ID: 6mY1mIoB1aJZhB_GIOt2
Text: Consect

## 4. Counting words and docs

`Elastic search` helps us to obtain the counts of words in each document. For example, the following code obtains the counts of words of a whole index by adding the counts of words obtained from each document through the functionality of `termvectors`. This function also allows us to get _document counts_ for computing tf-idf weights, by setting the `term_statistics` option to `True`.

In [ ]:
from elasticsearch.helpers import scan
from collections import Counter

# Search for all the documents and query the list of (word, frequency) of each one
# Totals are accumulated using a Counter for term frequencies, but not for document freqs.
word_counts = Counter()
doc_counts = Counter()
sc = scan(client, index='test', query={"query" : {"match_all": {}}})
for s in sc:
    tv = client.termvectors(index='test', id=s['_id'], fields=['text'], term_statistics=True, positions=False)
    if 'text' in tv['term_vectors']:   # just in case some document has no field named 'text'
        for t in tv['term_vectors']['text']['terms']:
            word = t
            count = tv['term_vectors']['text']['terms'][t]['term_freq']
            df = tv['term_vectors']['text']['terms'][t]['doc_freq']
            #pprint(tv['term_vectors']['text']['terms'][t])
            word_counts.update({word: count})
            doc_counts[word] = df      # the counts are not added to avoid overcounting

In [ ]:
# show word frequencies
word_counts.most_common()

[('consectetur', 28),
 ('sed', 27),
 ('ipsum', 23),
 ('labore', 23),
 ('eius', 22),
 ('est', 21),
 ('quisquam', 21),
 ('amet', 20),
 ('ut', 20),
 ('sit', 19),
 ('numquam', 18),
 ('dolorem', 18),
 ('etincidunt', 17),
 ('magnam', 16),
 ('voluptatem', 16),
 ('dolore', 15),
 ('velit', 15),
 ('aliquam', 14),
 ('non', 14),
 ('adipisci', 13),
 ('quiquia', 13),
 ('modi', 11),
 ('neque', 11),
 ('porro', 11),
 ('dolor', 10),
 ('quaerat', 9),
 ('tempora', 8)]

In [ ]:
# show doc freq
doc_counts.most_common()

[('amet', 10),
 ('consectetur', 10),
 ('est', 10),
 ('ipsum', 10),
 ('ut', 10),
 ('etincidunt', 9),
 ('labore', 9),
 ('neque', 9),
 ('numquam', 9),
 ('quiquia', 9),
 ('sed', 9),
 ('voluptatem', 9),
 ('eius', 8),
 ('quisquam', 8),
 ('non', 8),
 ('adipisci', 7),
 ('dolore', 7),
 ('magnam', 7),
 ('quaerat', 7),
 ('sit', 7),
 ('velit', 7),
 ('dolorem', 7),
 ('aliquam', 6),
 ('dolor', 6),
 ('porro', 6),
 ('tempora', 6),
 ('modi', 5)]

## 5. Proposed simple exercise

To get more familiar with elasticsearch, we propose that you _generate the Boolean and tf-idf matrices_ for the toy example that we used in class. You will find 7 text documents that contain the toy documents with the materials for this session in the racó. The steps to follow are:

- create an empty index
- open each text document in the `toy-docs` folder provided, read its contents and add it to the index as a new document; your index should contain 7 documents after this
- use the `termvectors` function to obtain term and doc counts, generate Boolean and tf-idf matrices based on these counts
- double check that your results coincide with the numbers in theory slides

## 6. Cleanup

Finally, we remove the test index..

In [ ]:
index.delete()

ObjectApiResponse({'acknowledged': True})